In [38]:
# using methods outlined in the assignment
from PIL import Image
import numpy as np
from scipy.signal import convolve2d

def pad_with_zeros(image, pad_width):
    return np.pad(image, pad_width, mode='constant', constant_values=0)


def apply_convolution(image, filter):
    return convolve2d(image, filter, mode='same', boundary='fill', fillvalue=0)


def energy_filter(image):
    # Sobel filters
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    # print("sobel_x: ", sobel_x)
    # Applying convolution
    Gx = apply_convolution(image, sobel_x)
    Gy = apply_convolution(image, sobel_y)
    # print("after convolution")
    # Gradient magnitude
    G = np.sqrt(Gx**2 + Gy**2)
    # print("after sqrt")
    # Edge angle and energy function
    edge_angle = np.arctan2(Gy, Gx)
    Ex = G * np.cos(edge_angle + np.pi / 2.0)
    Ey = G * np.sin(edge_angle + np.pi / 2.0)

    return Gx, Gy, G, Ex, Ey


def construct_graph_with_gradients(image, G):
    rows, cols = image.shape
    graph = {}

    for row in range(rows):
        for col in range(cols):
            graph[(row, col)] = {}
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    if dr == 0 and dc == 0:
                        continue 
                    nr, nc = row + dr, col + dc
                    if 0 <= nr < rows and 0 <= nc < cols:
                        # Use inverse of gradient magnitude as weight
                        weight = 1 / (G[nr, nc] + 1e-5)
                        graph[(row, col)][(nr, nc)] = weight

    return graph





In [41]:
import os
import time
import pandas as pd
from PIL import Image
import numpy as np
def process_images(folder_path):
    results = []

    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg','.bmp')):
            image_path = os.path.join(folder_path, filename)
            image = np.array(Image.open(image_path).convert('L'))
            print("image: ", image.shape)
            start_time = time.time()
            Gx, Gy, G, Ex,Ey = energy_filter(image)
            filter_time = time.time() - start_time
            print('Time Taken to Apply Filter',filter_time)
            start_time = time.time()
            graph = construct_graph_with_gradients(image, G)
            graph_time_taken = time.time() - start_time

            # Record the results
            results.append({
                'Image Name': filename,
                'Shape': image.shape,
                'Time Taken to Apply Filter': filter_time,
                'Time Taken to Construct Graph': graph_time_taken,
            })

    return pd.DataFrame(results)

folder_path = 'images' 
result_table = process_images(folder_path)
print(result_table)


image:  (2828, 4272)


KeyboardInterrupt: 

In [39]:
import sys
print(sys.path)

# from graphEnergy import  energy_filter, construct_graph_with_gradients
# simple tests to first test whether the filters work as expected
image = np.array([[ 57, 119, 242,  99, 120,  38,  89],
 [  3, 100, 210, 165, 186, 251, 255],
 [ 26, 125,   7, 178, 198, 154, 254],
 [193, 187, 251,  83,  98, 212,  61],
 [ 21,  99, 220, 237,  77, 205, 193],
 [ 31, 238,  96, 143, 190,  39,   4],
 [ 19, 201, 189,  19,  40,  18,  17]]) 
image2= np.array(
  [[10,20,51,61],[41,94,12,55],[52,22,31,81],[12,43, 68,30]]
)
Gx, Gy, G, Ex, Ey = energy_filter(image2)
print("Gx (Gradient in x-direction):\n", Gx)
print("\nGy (Gradient in y-direction):\n", Gy)
print("\nG (Gradient Magnitude):\n", G)
print("\nEx (Energy in x-direction):\n", Ex)
print("\nEy (Energy in y-direction):\n", Ey)

# note: result shows the same result as in the assignment, suggesting correct implementation

['/Users/kevin/Library/CloudStorage/OneDrive-UBC/school/2023 fall/520/COSC_520/testSuite2', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Users/kevin/Library/CloudStorage/OneDrive-UBC/school/2023 fall/520/520/lib/python3.8/site-packages']
Gx (Gradient in x-direction):
 [[-134  -53  -43  114]
 [-230   38  -22  106]
 [-181   15  -66  142]
 [-108  -91  -33  167]]

Gy (Gradient in y-direction):
 [[-176 -241 -173 -122]
 [ -86  -26   18  -20]
 [ 109   75  -36   -6]
 [ 126  127  165  193]]

G (Gradient Magnitude):
 [[221.20578654 246.75899173 178.26384939 166.97305172]
 [245.55243839  46.04345773  28.42534081 107.87029248]
 [211.2865353   76.4852927   75.17978452 142.12670404]
 [165.95180023 156.23699946 168.26764395 255.22147245]]

Ex (Energy in x-direction):
 [[ 176.  241.  173.  122.]
 [  86.   26.  -18.   20.]
 [-10